In [40]:
from keras.applications import vgg16
from keras.applications.vgg16 import preprocess_input
import cv2
import math
import numpy as np

def open_and_prepare_image(image_path, tile_size = 224, vertical_tiles = 3):
    image = cv2.imread(image_path)
    
    target_height = tile_size * vertical_tiles
    scale_percent = target_height/image.shape[0]
    
    horizontal_tiles = image.shape[1] * scale_percent // tile_size
    target_width = tile_size * horizontal_tiles
    
    scale_width = math.floor(scale_percent * image.shape[1])
    
    #print("(scale_width, target_height)", (scale_width, target_height))
    image = cv2.resize(image, (scale_width, target_height), interpolation=cv2.INTER_CUBIC)
    
    m1 = (image.shape[1] - image.shape[0])//2
    m2 = image.shape[0] + m1
    
    # print(m1, m2)
    
    return image[:,m1:m2,:]


def extract_windows(image, window_size=224, stride=32, channel_count=3):
    
    steps = ((image.shape[0] - window_size) // stride + 1, (image.shape[1] - window_size) // stride + 1)
    print(steps)
    
    windows = np.zeros((steps[0] * steps[1], window_size, window_size, channel_count))
    coords = np.zeros((steps[0] * steps[1], 2))

    # print("image.shape", image.shape)
        
    for i in range(steps[0]):
        for j in range(steps[1]):
            # print((stride*i),(stride*i+window_size), (stride*j),(stride*j+window_size))
            img = image[(stride*i):(stride*i+window_size), (stride*j):(stride*j+window_size)]
            windows[i * steps[1] + j] = img
            coords[i * steps[1] + j] = (i, j)

    return windows, coords


model = vgg16.VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
# model.summary()

image_path = "household_images/images/IMG_0175.jpg"

image = open_and_prepare_image(image_path)
windows, coords = extract_windows(image)

print(windows.shape)
x = preprocess_input(windows)
y = model.predict(x)
print(y.shape)
        
foo = {}

for i in range(21):
    for j in range(21):
        foo[(i,j)] = []
        
for i in range(y.shape[0]):
    coord = coords[i]
    for j in range(y.shape[1]):
        for k in range(y.shape[2]):
            foo[(int(coord[0] + j), int(coord[1] + k))].append(y[i,j,k,:])
            
# for i in range(21):
#     for j in range(21):
#         print((i,j), len(foo[(i,j)]))
        
print(foo[(6, 6)][0])
print(foo[(6, 6)][1])

(15, 15)
(225, 224, 224, 3)
(225, 7, 7, 512)
[ 0.          0.          0.          0.          0.          0.
  0.          3.1577945   0.          0.          0.          0.
  1.8961841   0.          0.          3.2635016   0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          1.4531853   0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          1.8277996  13.154027
  0.          0.          0.          0.          3.1747155   0.
  0.          6.6598334   0.42402926  0.          0.          0.
  0.          0.         16.391815    0.          0.          0.
  0.          0.          0.          0.          0.          0.
  1.7048279   0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0

In [ ]:
from matplotlib import pyplot as plt
img2 = windows[3][:,:,::-1]/255
plt.imshow(img2)

In [ ]:
for i in range(21):
    for j in range(21):
        foo[(i,j)] = []
        
for i in range(y.shape[0]):
    coord = coords[i]
    for j in range(y.shape[1]):
        for k in range(y.shape[2]):
            foo[(int(coord[0] + j), int(coord[1] + k))].append(y[i,j,k,:])
            
for i in range(21):
    for j in range(21):
        print((i,j), len(foo[(i,j)]))

In [ ]:
image.shape[0]//32